<a href="https://colab.research.google.com/github/GisstHubana/FHIR_Reporting_Tool/blob/GisstHubana-patch-html/Code/fhir_reporting_tool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vorbereitung
## CleanUp - Eventuell gespeicherte Variablen löschen vor Ausführung des Colab books für eine bessere Reproduzierbarkeit

In [1]:
%reset -f

## Installieren notwendiger Pakete

In [2]:
!pip install pyngrok
!pip install flask-ngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.7/718.7 kB 9.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-7.0.0-py3-none-any.whl size=21129 sha256=880f57fbac46bf96df6d43d8f278f0c9136d7a75cc1d423e374790b8bc8a7359
  Stored in directory: /root/.cache/pip/wheels/60/29/7b/f64332aa7e5e88fbd56d4002185ae22dcdc83b35b3d1c2cbf5
Successfully built pyngrok


## gnrok mit Authorizierungstoken verknüpfen

In [3]:
!ngrok authtoken 2Wlt2vGzytTpJ6zABKGKKwfiTof_7y4sewzGkNGy7zW6GB5oz

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


Importieren der notwendigen Libraries

In [4]:
from flask import Flask, request, render_template
from flask_ngrok import run_with_ngrok
import secrets
import sqlite3

Erstellung einer virtuellen Kopie des GitHub Repositorium FHIR_Reporting_Tool in Colab

In [5]:
%cd /content # Wechsel in den Ordner /content - für den Fall, dass in einem Unterordner gearbeitet wurde

!git clone https://github.com/GisstHubana/FHIR_Reporting_Tool.git #virtuelle Kopie des Repositoriums - Bei Neustart zuvor Runtime/disconnect and delete runtime ausführen
!ls # Auflistung aller Ordnerinhalte

[Errno 2] No such file or directory: '/content # Wechsel in den Ordner /content - für den Fall, dass in einem Unterordner gearbeitet wurde'
/content
Cloning into 'FHIR_Reporting_Tool'...
remote: Enumerating objects: 318, done.
remote: Counting objects: 100% (220/220), done.
remote: Compressing objects: 100% (194/194), done.
remote: Total 318 (delta 140), reused 23 (delta 23), pack-reused 98
Receiving objects: 100% (318/318), 71.41 KiB | 746.00 KiB/s, done.
Resolving deltas: 100% (172/172), done.
FHIR_Reporting_Tool  sample_data


### Database in Flask

#### Use database to match login credentials

In [6]:
db_folder = "/content/FHIR_Reporting_Tool/"
conn = sqlite3.connect(db_folder+"mydatabase.db") # falls Datenbank noch nicht existiert wird sie mit diesem Aufruf erzeugt
cursor = conn.cursor()

#query
create_table_query = """
CREATE TABLE IF NOT EXISTS users (
  id INTEGER PRIMARY KEY AUTOINCREMENT,
  username TEXT NOT NULL,
  password TEXT NOT NULL

)

"""
cursor.execute(create_table_query)

insert_query = """
INSERT INTO users(username, password)
SELECT ?, ?
WHERE NOT EXISTS (
  SELECT 1 FROM users WHERE username = ?

)
"""
user_data = ("USER3", "1234", "USER3")
cursor.execute(insert_query, user_data)
conn.commit()

select_query = "SELECT * FROM users"
cursor.execute(select_query)
rows = cursor.fetchall()
for row in rows:
  print(row)


(1, 'USER3', '1234')


In [9]:
template_folder = "/content/FHIR_Reporting_Tool/templates"
static_folder = "/content/FHIR_Reporting_Tool/static"
db_folder = "/content/FHIR_Reporting_Tool/"
app = Flask(__name__, template_folder=template_folder, static_folder=static_folder) # Flask-App, Benennung template-Ordner
app.secret_key = secrets.token_hex(32)
run_with_ngrok(app) # run_with_ngrok Funktion hinzufügen, um App mit ngrok auszuführen

conn = sqlite3.connect(db_folder+"mydatabase.db") # falls Datenbank noch nicht existiert wird sie mit diesem Aufruf erzeugt
cursor = conn.cursor()

@app.route('/query', methods=['POST'])
def query():
    query = request.form.get('query')
    cursor.execute(query)
    result = cursor.fetchall()
    return render_template('result_fhir.html', result=result)

@app.route('/') # Erstellung einer Routendefinition
def index():
    return render_template("index_fhir.html")

if __name__ == '__main__': # Starten der Flask-Anwendung
  app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://e3ef-34-80-167-108.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [03/Nov/2023 01:07:05] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/Nov/2023 01:07:06] "GET /favicon.ico HTTP/1.1" 404 -
ERROR:__main__:Exception on /query [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 2529, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1825, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1823, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1799, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "<ipython-input-9-371b0537880b>", line 14, in query
    cursor.execute(query)
sqlite3.ProgrammingError: SQLite objects created in a thread can only be used in that same thread. The objec

In [ ]:
template_folder = "/content/FHIR_Reporting_Tool/templates"
static_folder = "/content/FHIR_Reporting_Tool/static"
db_folder = "/content/FHIR_Reporting_Tool/"
app = Flask(__name__, template_folder=template_folder, static_folder=static_folder) # Flask-App, Benennung template-Ordner
# app.debug = True # Debug-Modus aktivieren - webseite lädt nicht wenn eingeschaltet
app.secret_key = secrets.token_hex(32)
run_with_ngrok(app) # run_with_ngrok Funktion hinzufügen, um App mit ngrok auszuführen

@app.route('/', methods=["GET", "POST"])
def login():

  if request.method == "POST":
    username = request.form.get("username")
    password = request.form.get("password")

    conn = sqlite3.connect(db_folder+"mydatabase.db") # falls Datenbank noch nicht existiert wird sie mit diesem Aufruf erzeugt
    cursor = conn.cursor()
    # query
    query = """
    SELECT * FROM users WHERE username = ? AND password = ?
    """
    cursor.execute(query, (username, password))
    result = cursor.fetchone()

    if result:
     return render_template("page2.html", name=username)
    else:
      message = "Wrong username or password. Please try again!"
      return render_template("login.html", message=message)

  else:
    return render_template("login.html")

@app.route('/home') # Erstellung einer Routendefinition home
def index():
    return render_template("index.html")

@app.route('/page2')
def page2():
    username = input("Enter your name:")
    return render_template("page2.html", name=username)

@app.route('/page3')
def page3():
    return render_template("page3.html")

app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://dd60-35-184-230-33.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [03/Nov/2023 00:51:58] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/Nov/2023 00:51:58] "GET /static/styles/myfile.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/Nov/2023 00:51:58] "GET /static/js/myjs.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/Nov/2023 00:51:58] "GET /favicon.ico HTTP/1.1" 404 -


## Abschluss - Runtime löschen und abschließend clear all outputs

In [ ]:
# Bei Neustart zuvor Runtime/disconnect and delete runtime ausführen
from google.colab import runtime
runtime.unassign() # delete runtime